# ChatBot Evaluation using Pytorch and Integrated Gradients

## Imports

In [1]:
import spacy
import pandas as pd
import numpy as np
from preprocess_data import raw_to_json, unzip_entities, tags_patterns_mix, remove_fallback, get_responses

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
!pip install captum

In [ ]:
from captum.attr import IntegratedGradients

## Data Preparation

In [3]:
df_train = pd.read_csv('data/Training - Training.csv')
df_test = pd.read_csv('data/Training - Test.csv')

In [4]:
# prepare training data
parent_tags = ['navigate', 'find', 'action']
intents = raw_to_json(df_train)
data = unzip_entities(intents, parent_tags)
tags_patterns = tags_patterns_mix(remove_fallback(data))

In [5]:
# prepare test data
df_test = df_test.drop(df_test[df_test['Tag'] == 'fallback'].index)
df_test.reset_index(inplace=True, drop=True)

In [6]:
responses = get_responses(data)

In [31]:
nlp = spacy.load("en_core_web_sm")
def lemmatizer(text):
    doc = nlp(text)
    return [d.lemma_ for d in doc]

def train_pipeline(tags_patterns):
    
    word_list = []
    tags = []
    word_tag_data = []
    
    for i, row in tags_patterns.iterrows():
        tag = row['tag']
        pattern = row['pattern']
        tags.append(tag)
        word = lemmatizer(pattern)
        word_list.extend(word)
        word_tag_data.append((word, tag))
        
    return tags, word_list, word_tag_data

def prepare_training_data(word_tag_data, word_list, tags):
    X = []
    y = []
    
    for (pattern, tag) in word_tag_data:
        bog = bag_of_words(pattern, word_list)
        X.append(bog)
        label = tags.index(tag)
        y.append(label)

    return np.array(X), np.array(y)

def bag_of_words(tokenized_sentence, words):

    bog = np.zeros(len(words), dtype=np.float32)
    for idx, word in enumerate(words):
        if word in tokenized_sentence:
            bog[idx] = 1
    return bog

def pipe_new_input(text):
    if text == '': #otherwise error
        text = ' '
    text = lemmatizer(text)
    bog = bag_of_words(text, word_list)
    x = bog.reshape(1, bog.shape[0])
    return x
    
def get_tag_from_prediction(result, tags, threshold, fallback = 'fallback'):
    
    max_proba = torch.max(result)
    
    if max_proba < threshold:
        return fallback
    
    predicted_tag = tags[torch.argmax(result)]
    
    return predicted_tag
    
def test(df, model, tags, threshold = 0.4, col_name = 'Sentence', col_tag = 'Tag'):
    predicted_tags = []
    matches = []
    probas = []
    predicted_tags_if_not_fallback = []
    
    for i, row in df.iterrows():
        sentence = row[col_name]
        tag = row[col_tag]

        text = lemmatizer(sentence)
        bog = bag_of_words(text, word_list) # word_list is "global"
        x = bog.reshape(1, bog.shape[0])
        x = torch.from_numpy(x)
        result = model(x)

        max_proba = torch.max(result[0])
        
        predicted_tag = get_tag_from_prediction(result, tags, threshold)
        
        predicted_tag_except_fallback = get_tag_from_prediction(result, tags, 0.0)
        
        predicted_tags_if_not_fallback.append(predicted_tag_except_fallback)
        probas.append(max_proba)
        predicted_tags.append(predicted_tag)
        matches.append(predicted_tag == tag)

    df = df.assign(predicted_tags = predicted_tags, matches = matches, probas = probas, predicted_tags_if_not_fallback = predicted_tags_if_not_fallback)
    
    acc = df['matches'].sum() / len(df)
    output = f'Accuracy: {acc}'
    
    return df, output

In [8]:
tags, word_list, word_tag_data = train_pipeline(tags_patterns)
IGNORE = ['?', '!', '.', ',']
word_list = [word for word in word_list if word not in IGNORE]
word_list = sorted(set(word_list))
tags = sorted(set(tags))
X, y = prepare_training_data(word_tag_data, word_list, tags)

In [9]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)
y = F.one_hot(y)
y = y.type(torch.FloatTensor)

## Model Definition

In [10]:
device = torch.device('cpu')

In [11]:
X.shape, y.shape

(torch.Size([1137, 154]), torch.Size([1137, 30]))

Input Size: 154

Output Size: 30 (all possible tags)

Hidden Layer Size: 16 (self defined)

In [75]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(154, 16),
        nn.ReLU(),
        nn.Linear(16, 30),
        nn.Sigmoid()
    )
    self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
    self.loss_function = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.layers(x)

  def train(self, X, y):
    self.optimizer.zero_grad()
    output = self(X)
    loss = self.loss_function(output, y)
    loss.backward()
    self.optimizer.step()
    return loss.item()

In [76]:
mlp = MLP().to(device)

## Training

In [77]:
for epoch in range(0, 50):
  print(f'Epoch: {epoch}')
  current_loss = 0.0
  for a in range(2):
    for i in range(len(X)):
      loss = mlp.train(torch.unsqueeze(X[i], dim=0), torch.unsqueeze(y[i], dim=0))
      current_loss += loss
      if (i % 100) == 0:
        print(f'Loss after {i} iterations: {current_loss}')
        current_loss = 0.0

Epoch: 0
Loss after 0 iterations: 3.368842601776123
Loss after 100 iterations: 336.77941393852234
Loss after 200 iterations: 339.8909981250763
Loss after 300 iterations: 342.05718302726746
Loss after 400 iterations: 341.2761664390564
Loss after 500 iterations: 340.630264043808
Loss after 600 iterations: 342.6890251636505
Loss after 700 iterations: 341.31024074554443
Loss after 800 iterations: 336.8361930847168
Loss after 900 iterations: 340.4233407974243
Loss after 1000 iterations: 342.4567756652832
Loss after 1100 iterations: 340.7729856967926
Loss after 0 iterations: 127.13477897644043
Loss after 100 iterations: 335.2034683227539
Loss after 200 iterations: 338.25338315963745
Loss after 300 iterations: 339.2520761489868
Loss after 400 iterations: 338.38466477394104
Loss after 500 iterations: 336.8469581604004
Loss after 600 iterations: 340.71096181869507
Loss after 700 iterations: 337.6544828414917
Loss after 800 iterations: 332.8676838874817
Loss after 900 iterations: 338.00570058822

## Evaluation

In [78]:
with torch.no_grad():
  df_test_result, acc = test(df_test, mlp, tags, col_name = 'Sentence', col_tag = 'Tag', threshold = 0.25)

In [79]:
acc

'Accuracy: 0.6666666666666666'

In [80]:
df_test_result

,Sentence,Tag,Tag_parent,predicted_tags,matches,probas,predicted_tags_if_not_fallback
0,Where can I unfollow another person?,find_follow,find,find_follow,True,tensor(0.9996),find_follow
1,How to get to the homepage?,find_homepage,find,find_homepage,True,tensor(0.9979),find_homepage
2,Where do I find my profile?,find_profile,find,find_profile,True,tensor(0.9986),find_profile
3,Where can I view my feed?,find_personal_feed,find,find_personal_feed,True,tensor(0.9944),find_personal_feed
4,Where can I access the public feed?,find_public_feed,find,find_public_feed,True,tensor(0.9984),find_public_feed
5,Where can I change my name?,find_edit_profile,find,find_edit_profile,True,tensor(0.9973),find_edit_profile
6,Where can I login?,find_auth,find,find_auth,True,tensor(0.9994),find_auth
7,Where can I change the language to german?,find_language,find,find_language,True,tensor(1.),find_language
8,How do I write a post?,find_create_post,find,find_create_post,True,tensor(0.9979),find_create_post
9,What do I have to do to publish my own trade?,find_create_trade,find,find_create_trade,True,tensor(0.9989),find_create_trade


## Explanation

In [182]:
ig = IntegratedGradients(mlp)

In [193]:
def explain_choice(ig, text):
  x = pipe_new_input(text)
  x = torch.from_numpy(x)
  with torch.no_grad():
    outputs = mlp(x)
    target = torch.argmax(outputs) # is prediction not actual
    print(tags[target])
    # apply integrated gradients and receive importance of input nodes
    outputs = ig.attribute(
        inputs=x,
        target=target,
        baselines=torch.unsqueeze(torch.zeros(154), dim=0),
        n_steps=50)
  sorted, indices = torch.sort(torch.absolute(outputs), descending=True)
  sorted, indices = sorted[0].tolist(), indices[0].tolist()

  s = []
  i = []

  for ind, value in enumerate(sorted):
    if not (value > 0):
      break
    i.append(word_list[indices[ind]])
    s.append(value)
  print(list(zip(s, i)))

In [194]:
# wrong prediction
text = 'I would like to create a new post.'
explain_choice(ig, text)

navigate_create_post
[(0.6177329100201314, 'post'), (0.18393473152185524, 'new'), (0.17071351897738424, 'create'), (0.1499542321187939, 'like'), (0.13358355625404758, '-PRON-'), (0.10438274771541241, 'would'), (0.07845328218236938, 'a'), (0.037957520685217266, 'to')]


In [197]:
# wrong prediction
text = 'Create a post.'
explain_choice(ig, text)

navigate_create_post
[(0.8853112790963429, 'post'), (0.22318878729466662, 'create'), (0.118740701145351, 'a')]


In [198]:
# correct prediction
text = 'Create a post for me.'
explain_choice(ig, text)

action_create_post
[(0.6527565482046565, 'post'), (0.2666675698634774, 'create'), (0.1877163463185576, 'for'), (0.1419184454426143, '-PRON-'), (0.01056351783490079, 'a')]
